# Markov chain from data

In this notebook we will load some dummy data and decompose it using the Kemeny Decomposition Algorithm (KDA).

## Installing `pykda`

Running the following command installs `pykda` in Google Colab (note that this may take a while as it also installs some other packages). If you run this notebook in your local environment which already has `pykda` installed, you can skip this step.

In [1]:
!pip install pykda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 949.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.0 MB/s eta 0:00:00
  Created wheel for tarjan: filename=tarjan-0.2.4-py3-none-any.whl size=7974 sha256=75b28fd48c61f65738df8be0cf94a9faf69c42a5e80d4077a210eb61f4e91e6f
  Stored in directory: /root/.cache/pip/wheels/5e/43/39/663420cf5f9f64699d1889da5e52f2b26b33dd603e53120a8a
Successfully built tarjan
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


## Load data as a Markov chain
First let us load some dummy data that consists of 4 data points (each row consists of a data point).

In [11]:
import numpy as np

data = np.array(
    [
        [0, 0],
        [0.2, .1],
        [-0.1, .3],
        [1.1, 1.3],
        [1.2, 0.9],                    
        ]
    )

We can calculate the transition matrix from the data using the Gaussian similarity matrix that calculates the similarity between two data points as a Gaussian function of the Euclidean distance between them. The Markov chain transition matrix is then calculated as the row-normalized similarity matrix.

In [13]:
from pykda.Markov_chain import MarkovChain
from pykda.loaders import load_from_data

MC = MarkovChain(load_from_data(data))
print('Markov chain information:')
print(MC)
print('Transition matrix = \n', MC.P)

Markov chain information:
MC with 5 states.
Ergodic classes: [[4, 3, 2, 1, 0]].
Transient classes: [].
Transition matrix = 
 [[0.35045276 0.32626018 0.30373767 0.00553141 0.01401799]
 [0.32135699 0.34518599 0.28660455 0.01380733 0.03304514]
 [0.31168211 0.29858819 0.35961906 0.01096101 0.01914965]
 [0.00843874 0.02138592 0.01629594 0.53465247 0.41922693]
 [0.02027258 0.04851847 0.026988   0.3974022  0.50681875]]
Transition matrix = 
 [[ 4.33672204  4.02307089  4.58158357 37.9037732  33.21039846]
 [ 4.83323392  4.27154779  5.01717769 37.03506039 32.33712179]
 [ 4.6522367   4.27766779  4.45015156 37.55569345 32.89548394]
 [18.31062193 16.6317461  17.89188906  6.61612474  5.38925362]
 [17.51229809 15.82885839 17.12673044  9.28430452  6.27169281]]


By looking at the mean first passage matrix in the following, we already see that there is a natural decomposition in the Markov chain corresponding to the first 3 data points and the last 2 data points.

In [14]:
print(MC.mean_first_passage_matrix)


[[ 4.33672204  4.02307089  4.58158357 37.9037732  33.21039846]
 [ 4.83323392  4.27154779  5.01717769 37.03506039 32.33712179]
 [ 4.6522367   4.27766779  4.45015156 37.55569345 32.89548394]
 [18.31062193 16.6317461  17.89188906  6.61612474  5.38925362]
 [17.51229809 15.82885839 17.12673044  9.28430452  6.27169281]]


## Decomposing the Markov chain

We now apply the Kemeny decomposition algorithm to decompose the Markov chain. We will remove all edges with negative Kemeny constant derivatives at once (in their notation, we use KDA(P, CO_A_2(3), CO_B_1(1), FALSE) where P is the above transition matrix). This identifies the natural decomposition of the Markov chain into two components by looking at the weakly connected components of the resulting Markov chain.

In [20]:
from pykda.KDA import KDA
from IPython.display import HTML

kda = KDA(original_MC=MC, CO_A="CO_A_1(1)", CO_B="CO_B_3(0)", symmetric_cut=False)
kda.run()
name = 'data_example_KDA_A1_1_B3_0'
kda.plot(file_name=name, notebook=False)
HTML(name + '.html')
print('The weakly connected components after KDA are: ', kda.MC.weakly_connected_components)

data_example_KDA_A1_1_B3_0.html
The weakly connected components after KDA are:  [[2, 1, 0], [4, 3]]


## Other ways of normalizing the Gaussian similarity matrix

There are other ways of normalizing the Gaussian similarity matrix. For example, we can first ensure all rows sums are equal by adding (stronger) self-loops. Alternatively, we can add an extra node/state to ensure that the stationary distribution of the Markov chain after the normalization is proportional to the eigenvector centrality of the Gaussian similarity matrix (of the original nodes). Both are illustrates in the following, respectively.

In [31]:
from pykda.normalizers import *
from pykda.utilities import eigenvec_centrality, Gaussian_similarity

MC_self_loop = MarkovChain(load_from_data(data, normalization_with_self_loops))
print('Transition matrix by adding self-loops:')
print(MC_self_loop.P.round(3))
MC_eig_vec_centr = MarkovChain(load_from_data(data, normalization_same_eigenvec_centr))
print('Transition matrix with same eigenvector centrality:')
print(MC_eig_vec_centr.P.round(3))
 

Transition matrix by adding self-loops:
[[0.36  0.321 0.299 0.005 0.014]
 [0.321 0.345 0.287 0.014 0.033]
 [0.299 0.287 0.385 0.011 0.018]
 [0.005 0.014 0.011 0.7   0.271]
 [0.014 0.033 0.018 0.271 0.664]]
Transition matrix with same eigenvector centrality:
[[0.    0.31  0.307 0.298 0.037 0.049]
 [0.015 0.345 0.321 0.299 0.005 0.014]
 [0.    0.321 0.345 0.287 0.014 0.033]
 [0.04  0.299 0.287 0.345 0.011 0.018]
 [0.354 0.005 0.014 0.011 0.345 0.271]
 [0.319 0.014 0.033 0.018 0.271 0.345]]


Let us check that the stationary distribution of the (last) Markov chain after the normalization is proportional to the eigenvector centrality of the Gaussian similarity matrix (of the original nodes).

In [30]:
print('Stationary distribution of the Markov chain with same eigenvector centrality:')
stat_distr_orig_nodes = MC_eig_vec_centr.stationary_distribution[1:]
print(stat_distr_orig_nodes / sum(stat_distr_orig_nodes))
print('Eigenvector centrality of the Gaussian similarity matrix:')
eig_vec = eigenvec_centrality(Gaussian_similarity(data))[0]
print(eig_vec / sum(eig_vec))

Stationary distribution of the Markov chain with same eigenvector centrality:
[[0.30951835]
 [0.3068301 ]
 [0.2982355 ]
 [0.0365545 ]
 [0.04886155]]
Eigenvector centrality of the Gaussian similarity matrix:
[[0.30951835]
 [0.3068301 ]
 [0.2982355 ]
 [0.0365545 ]
 [0.04886155]]
